In [1]:
!pip install transformers datasets evaluate rouge_score accelerate

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=83b22702143d318c144ca347397581a3739aae78f2d780d5dbead53502aa5cee
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
import transformers
transformers.__version__

'4.44.2'

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from huggingface_hub import list_datasets

datasets_list = list_datasets()

num = 0
for dataset in datasets_list:
  print(dataset)

  if (num > 10):
    break

  num += 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetInfo(id='KingNish/reasoning-base-20k', author='KingNish', sha='ae93576e3b315cf876e7429b7fa1fd041df72d29', created_at=datetime.datetime(2024, 10, 2, 15, 9, 22, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 10, 5, 14, 19, 30, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=365, downloads_all_time=None, likes=55, paperswithcode_id=None, tags=['task_categories:text-generation', 'language:en', 'license:apache-2.0', 'size_categories:10K<n<100K', 'format:json', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'region:us', 'reasoning', 'synthetic'], card_data=None, siblings=None)
DatasetInfo(id='fka/awesome-chatgpt-prompts', author='fka', sha='459a66186f8f83020117b8acc5ff5af69fc95b45', created_at=datetime.datetime(2022, 12, 13, 23, 47, 45, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 9, 3, 21, 28, 41, tzinfo=datetime.timezone.utc), private=False, gated=False

In [5]:
from datasets import load_dataset

billsum_ds = load_dataset("FiscalNote/billsum")
billsum_ds

README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
})

In [6]:
billsum_ds = load_dataset("FiscalNote/billsum", split="train[:1%]")
billsum_ds

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 189
})

In [7]:
billsum_ds=billsum_ds.train_test_split(test_size=0.2, seed=42)
billsum_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 151
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 38
    })
})

In [8]:
def clean_txt(example):
    for txt in ["summary", "title"]:
       example[txt]  = example[txt].lower()
       example[txt]  = example[txt].replace("/", "")
       example[txt]  = example[txt].replace("\n", "")
       example[txt]  = example[txt].replace("``", "")
       example[txt]  = example[txt].replace('"', '')
       example[txt]  = example[txt].replace("--", "")

    return example

In [18]:
billsum_ds = billsum_ds.map(clean_txt)
billsum_ds

Map:   0%|          | 0/151 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 151
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 38
    })
})

In [19]:
billsum_ds['train']["summary"][7]

'emergency malpractice liability insurance commission (emlic) act - establishes the emergency malpractice liability insurance commission, whose purpose is to examine the causes of soaring medical malpractice premiums and propose a comprehensive strategy to combat the consequences. directs the commission to investigate the possible linkage between: (1) skyrocketing malpractice insurance premiums; (2) rising jury awards; (3) decreased accessibility and affordability of health care; and (4) a rise in the number of physicians moving, quitting, or retiring.'

In [20]:
 billsum_ds['train']["summary"][0]

'ravi thackurdeen safe students study abroad act this bill amends title iv (student assistance) of the higher education act of 1965 to modify campus security reporting requirements for an institution of higher education (ihe) that participates in federal student aid programs. currently, an ihe must annually report data to the department of education on certain criminal offenses that occur in the following geographic categories: on campus, on campus in a residential facility, on noncampus property, and on public property. this bill expands the geographic categories of reportable offenses to also include crimes that occur while a student is participating in an approved study abroad program.  additionally, it requires an ihe to develop and distribute, as part of its annual security report provided to students and employees, a statement that it has adopted and implemented a program to protect students participating in an approved study abroad program from crime and harm.'

In [21]:
 billsum_ds['train']["summary"][0]

'ravi thackurdeen safe students study abroad act this bill amends title iv (student assistance) of the higher education act of 1965 to modify campus security reporting requirements for an institution of higher education (ihe) that participates in federal student aid programs. currently, an ihe must annually report data to the department of education on certain criminal offenses that occur in the following geographic categories: on campus, on campus in a residential facility, on noncampus property, and on public property. this bill expands the geographic categories of reportable offenses to also include crimes that occur while a student is participating in an approved study abroad program.  additionally, it requires an ihe to develop and distribute, as part of its annual security report provided to students and employees, a statement that it has adopted and implemented a program to protect students participating in an approved study abroad program from crime and harm.'

In [22]:
EXAMPLE_TEXT_INDEX = 0
example_text = billsum_ds["test"]["summary"][EXAMPLE_TEXT_INDEX]
example_text

'stop abuse, violence, and exploitation of elders act of 2012 or the save elders act of 2012 - amends the victims of crime act of 1984 to designate specified funds from the crime victims fund for grants for compensation and assistance to victims of elder abuse.defines elder abuse under that act: (1) to mean the abuse, exploitation, or neglect of an individual who is age 65 or older and lawfully present in the united states; and (2) with respect to a state that receives a grant, to include any other conduct that is defined as such under the laws of the state.requires the director of the office of victims of crime to use such funds to make grants to states to support: (1) eligible crime victim assistance programs that provide assistance to victims of elder abuse; and (2) programs that improve the investigation, handling, and prosecution of cases of elder abuse.apportions specified percentages of grant amounts among the states, the district of columbia, puerto rico, guam, the u.s. virgin 

In [23]:
MODEL_NAME= "google/pegasus-large"

In [24]:
from transformers import pipeline

prefix = "summarize: "

summarizer = pipeline("summarization", model = MODEL_NAME)

summary_txt = summarizer(prefix + example_text)

summary_txt

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'summary_text': 'virgin islands, american samoa, and the northern mariana islands and requires the amounts remaining after such apportionment to be distributed among the states based on: (1) the population of individuals age 65 or older in a state relative to the population of such individuals in all states, (2) the population of individuals age 65 or older in a state in relation to the population of the state, and (3) the overall rate of crime in a state.'}]

In [25]:
summarizer = pipeline("summarization", model = MODEL_NAME, truncation=True)
summary_txt = summarizer(prefix + example_text)
summary_txt

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'summary_text': 'virgin islands, american samoa, and the northern mariana islands and requires the amounts remaining after such apportionment to be distributed among the states based on: (1) the population of individuals age 65 or older in a state relative to the population of such individuals in all states, (2) the population of individuals age 65 or older in a state in relation to the population of the state, and (3) the overall rate of crime in a state.'}]

In [26]:
ref_txt = billsum_ds["test"]["title"][EXAMPLE_TEXT_INDEX]
ref_txt

'a bill to dedicate funds from the crime victims fund to victims of elder abuse, and for other purposes.'

In [27]:
import evaluate
rouge = evaluate.load("rouge")
rouge

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [28]:
reference = "Albrechts karriere ble definert av hans store engasjement for sjeldent fremført repertoar, noe som ga ham et rykte for å gjenopplive skjulte musikalske perler."

ex_1 = "Albrechts dedikasjon til å avdekke skjulte musikalske perler var tydelig i hans store engasjement for sjeldent fremført repertoar, og brakte glemte stykker til live på scenen."

ex_2 = "Hans lidenskap for mangfold i klassisk musikkprogrammering førte til at Albrecht fokuserte på sjeldent fremført repertoar, og ga publikum en unik lytteopplevelse."


In [29]:
results = rouge.compute(predictions = [ex_1], references = [reference])
results

{'rouge1': 0.48000000000000004,
 'rouge2': 0.37499999999999994,
 'rougeL': 0.35999999999999993,
 'rougeLsum': 0.35999999999999993}

In [30]:
results = rouge.compute(predictions = [ex_2], references = [reference])
results

{'rouge1': 0.2916666666666667,
 'rouge2': 0.13043478260869565,
 'rougeL': 0.2916666666666667,
 'rougeLsum': 0.2916666666666667}

In [31]:
summary_txt[0]["summary_text"]

'virgin islands, american samoa, and the northern mariana islands and requires the amounts remaining after such apportionment to be distributed among the states based on: (1) the population of individuals age 65 or older in a state relative to the population of such individuals in all states, (2) the population of individuals age 65 or older in a state in relation to the population of the state, and (3) the overall rate of crime in a state.'

In [32]:
result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                       references = [ref_txt], use_stemmer = True)
result

{'rouge1': 0.14583333333333334,
 'rouge2': 0.0,
 'rougeL': 0.12499999999999999,
 'rougeLsum': 0.12499999999999999}

In [33]:
article_texts = billsum_ds["test"]["title"]

article_categories = billsum_ds["test"]["summary"]

In [34]:
from tqdm import tqdm

candidate_summaries = []

prefix = "summarize: "

for i, text in enumerate(tqdm(article_texts[:50])):

    candidate = summarizer(prefix + text)

    candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 38/38 [07:35<00:00, 11.98s/it]


In [35]:
result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_categories[:50],
                           use_stemmer = True)
result_agg

{'rouge1': 0.17834663639333276,
 'rouge2': 0.10801851697806446,
 'rougeL': 0.15333180609342884,
 'rougeLsum': 0.15206591042863848}

In [36]:
result_unagg = rouge.compute(predictions = candidate_summaries,
                             references = article_categories[:50],
                             use_stemmer = True,
                             use_aggregator = False)
result_unagg

{'rouge1': [0.11940298507462686,
  0.1647940074906367,
  0.10256410256410256,
  0.10869565217391304,
  0.10576923076923077,
  0.02102496714848883,
  0.1016949152542373,
  0.14168937329700274,
  0.029850746268656723,
  0.03265306122448979,
  0.16037735849056603,
  0.06086956521739129,
  0.09252669039145907,
  0.5641025641025641,
  0.07523510971786834,
  0.049019607843137254,
  0.05494505494505494,
  0.21428571428571427,
  0.11733333333333333,
  0.39705882352941174,
  0.30588235294117644,
  0.05813953488372092,
  0.2213740458015267,
  0.336283185840708,
  0.13513513513513511,
  0.07042253521126761,
  0.05405405405405405,
  0.6222222222222221,
  0.14130434782608695,
  0.1865671641791045,
  0.05555555555555556,
  0.4888888888888889,
  0.2280701754385965,
  0.20392156862745098,
  0.22950819672131148,
  0.48611111111111116,
  0.08759124087591241,
  0.1515151515151515],
 'rouge2': [0.08270676691729323,
  0.09056603773584905,
  0.08695652173913043,
  0.08888888888888889,
  0.0676328502415459,


In [37]:
import numpy as np

result_unagg_rsum = np.array(result_unagg["rouge2"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(27, 8)

In [38]:
import pandas as pd

act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_categories[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,summarize: a bill to dedicate funds from the c...,"stop abuse, violence, and exploitation of elde..."
1,summarize: to provide for additional benefits ...,safety net extension act - amends the temporar...
2,summarize: gallatin land consolidation act of ...,gallatin land consolidation act of 1998 - prov...
3,summarize: women's progress commemoration act,women's progress commemoration act - establish...
4,summarize: to amend the small business investm...,premier certified lenders program improvement ...


In [39]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(20, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(20, "Reference_summaries"))

Actual Summary
summarize: to amend the internal revenue code of 1986 to provide a federal income tax credit for certain home purchases.
Reference Summary
helping homebuyers act of 2009 - amends the internal revenue code to allow individual taxpayers a one-time tax credit for 15% of the purchase price of a principal residence, up to $18,000. requires repayment of such credit if the taxpayer sells the residence, or fails to occupy the residence as a principal residence, at any time within 60 months after the date of purchase.


In [40]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(18, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(18, "Reference_summaries"))

Actual Summary
summarize: to amend part b of title xviii of the social security act to assure access to durable medical equipment under the medicare program.
Reference Summary
medicare durable medical equipment access act of 2005 - amends title xviii (medicare) of the social security act to prohibit the secretary of health and human services from awarding any contracts under the competitive durable medical equipment items and services acquisition program unless: (1) the quality standards have been implemented; and (2) the secretary has received advice from the program oversight committee.requires the secretary (who currently is authorized), in carrying out competitive acquisition programs, to exempt: (1) rural areas and areas with low population density within urban areas that are not competitive, unless there is a significant national market through mail order for a particular item or service; and (2) items and services for which the application of competitive acquisition is not likel

In [41]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [42]:
tokenizer(["four-time defending champion"])

{'input_ids': [[541, 121, 1139, 10838, 6232, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1]]}

In [43]:
tokens = tokenizer.tokenize("four-time defending champion")
tokens


['▁four', '-', 'time', '▁defending', '▁champion']

In [44]:
ids = tokenizer.convert_tokens_to_ids(tokens)
ids

[541, 121, 1139, 10838, 6232]

In [45]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
prefix = "summarize: "


def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples["summary"]]
    model_inputs = tokenizer(inputs, max_length = 512, truncation = True)

    labels = tokenizer(text_target = examples["title"], max_length = 128, truncation = True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [47]:
tokenized_billsum_ds = billsum_ds.map(preprocess_function, batched = True)

tokenized_billsum_ds

Map:   0%|          | 0/151 [00:00<?, ? examples/s]

Map:   0%|          | 0/38 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 151
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 38
    })
})

In [48]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = MODEL_NAME)

In [49]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    result = rouge.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)

    prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["generated_length"] = np.mean(prediction_length)

    return {k: round(v, 4) for k, v in result.items()}

In [50]:
from huggingface_hub import notebook_login
notebook_login()

In [53]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "Fine-Tuned-billsum-Summarization",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 2,  # Reduced batch size
    per_device_eval_batch_size = 2,  # Reduced batch size
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 3,
    predict_with_generate = True,
    fp16 = True,  # Enable mixed precision training if supported
    push_to_hub = True,
    gradient_accumulation_steps = 4, # accumulate gradients over 4 steps
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_billsum_ds["train"],
    eval_dataset = tokenized_billsum_ds["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

# Use gradient accumulation to simulate a larger batch size without increasing memory usage
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Generated Length
1,No log,2.300256,0.221800,0.094600,0.180700,0.179200,106.578900
2,No log,1.973429,0.216400,0.092200,0.179900,0.178900,104.447400
3,No log,1.861797,0.216900,0.091600,0.180900,0.179900,104.815800


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


TrainOutput(global_step=57, training_loss=2.33529556006716, metrics={'train_runtime': 568.6767, 'train_samples_per_second': 0.797, 'train_steps_per_second': 0.1, 'total_flos': 362599566557184.0, 'train_loss': 2.33529556006716, 'epoch': 3.0})

In [54]:
trainer.push_to_hub()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


events.out.tfevents.1728478443.3b1d326743dc.296.2:   0%|          | 0.00/9.47k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tahreem636/Fine-Tuned-billsum-Summarization/commit/5030d86d49c996d81453c5d11ebd45ef2c163c19', commit_message='End of training', commit_description='', oid='5030d86d49c996d81453c5d11ebd45ef2c163c19', pr_url=None, pr_revision=None, pr_num=None)